In [1]:
#Check qiime version and path
!which qiime

/space/home/rosales/anaconda2/envs/qiime2-2018.11/bin/qiime


In [2]:
#mv to working directory
!pwd
%cd /space/home/rosales/projects/2019_Acer_nutrients/
!pwd

/space/home/rosales/projects/2019_Acer_nutrients/notebooks
/space/home/rosales/projects/2019_Acer_nutrients
/space/home/rosales/projects/2019_Acer_nutrients


In [5]:
#import only forward reads
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path /space/home/rosales/rawdata/2019_Mar_Acer_nutr_tank_rsmas/CoralNutrientMB/Forward_reads/ \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-single-end.qza

Imported /space/home/rosales/rawdata/2019_Mar_Acer_nutr_tank_rsmas/CoralNutrientMB/Forward_reads/ as CasavaOneEightSingleLanePerSampleDirFmt to demux-single-end.qza


In [8]:
!qiime cutadapt trim-single \
--i-demultiplexed-sequences demux-single-end.qza \
--p-cores 16 \
--p-front GTGYCAGCMGCCGCGGTAA \
--o-trimmed-sequences demux-single-end_trim.qza

Saved SampleData[SequencesWithQuality] to: demux-single-end_trim.qza


In [9]:
%time
!qiime dada2 denoise-single \
--i-demultiplexed-seqs /space/home/rosales/projects/2019_Acer_nutrients/demux-single-end_trim.qza \
--p-trunc-len 220 \
--p-trim-left 20 \
--p-n-threads 20  \
--o-representative-sequences /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_AcNu_trim.qza \
--o-table /space/home/rosales/projects/2019_Acer_nutrients/table_AcNu_trim.qza \
--o-denoising-stats /space/home/rosales/projects/2019_Acer_nutrients/stats_dada_AcNu_trim


CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 15.5 µs
Saved FeatureTable[Frequency] to: /space/home/rosales/projects/2019_Acer_nutrients/table_AcNu_trim.qza
Saved FeatureData[Sequence] to: /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_AcNu_trim.qza
Saved SampleData[DADA2Stats] to: /space/home/rosales/projects/2019_Acer_nutrients/stats_dada_AcNu_trim.qza


In [10]:
#used a trained silva db base for taxonomy
%time
!qiime feature-classifier classify-sklearn \
  --i-classifier /space/home/rosales/databases/silva-132-99-515-806-nb-classifier.qza \
  --i-reads /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_AcNu_trim.qza \
  --p-n-jobs -5 \
  --o-classification /space/home/rosales/projects/2019_Acer_nutrients/taxonomyAcNu_trim.qza

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.9 µs
Saved FeatureData[Taxonomy] to: /space/home/rosales/projects/2019_Acer_nutrients/taxonomyAcNu_trim.qza


In [11]:
# Retain only bacteria and archaea in ASV table
%time
!qiime taxa filter-table \
  --i-table /space/home/rosales/projects/2019_Acer_nutrients/table_AcNu_trim.qza \
  --i-taxonomy /space/home/rosales/projects/2019_Acer_nutrients/taxonomyAcNu_trim.qza \
  --p-exclude mitochondria,chloroplast \
  --p-include D_0__Bacteria,D_0__Archaea \
  --o-filtered-table /space/home/rosales/projects/2019_Acer_nutrients/table-BacArc_AcNu_trim.qza


CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 12.2 µs
Saved FeatureTable[Frequency] to: /space/home/rosales/projects/2019_Acer_nutrients/table-BacArc_AcNu_trim.qza


In [12]:
# Retain only bacteria and archaea sequences

!qiime taxa filter-seqs \
  --i-sequences /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_AcNu_trim.qza\
  --i-taxonomy  /space/home/rosales/projects/2019_Acer_nutrients/taxonomyAcNu_trim.qza \
  --p-exclude mitochondria,chloroplast \
  --p-include D_0__Bacteria,D_0__Archaea \
  --o-filtered-sequences /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_BacArc_AcNu_trim.qza


Saved FeatureData[Sequence] to: /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_BacArc_AcNu_trim.qza


In [ ]:
#get metadata file
!git clone https://github.com/srosales712/AcNutrients.git

In [13]:
#visualize table
!qiime feature-table summarize \
  --i-table /space/home/rosales/projects/2019_Acer_nutrients/table-BacArc_AcNu_trim.qza \
  --o-visualization /space/home/rosales/projects/2019_Acer_nutrients/table-BacArc_AcNu_trim.qzv\
  --m-sample-metadata-file /space/home/rosales/projects/2019_Acer_nutrients/AcNutrients/Nutrient_16s_Metadata.txt

Saved Visualization to: /space/home/rosales/projects/2019_Acer_nutrients/table-BacArc_AcNu_trim.qzv


In [14]:
#visualize reads
!qiime feature-table tabulate-seqs \
--i-data /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_BacArc_AcNu_trim.qza \
--o-visualization /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_BacArc_AcNu_trim.qzv


Saved Visualization to: /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_BacArc_AcNu_trim.qzv


In [15]:
#align sequences 
%time
!qiime alignment mafft \
  --i-sequences /space/home/rosales/projects/2019_Acer_nutrients/rep_seqs_BacArc_AcNu_trim.qza \
  --o-alignment /space/home/rosales/projects/2019_Acer_nutrients/aligned_rep_seqs_BacArc_AcNu_trim.qza

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 12.2 µs
Saved FeatureData[AlignedSequence] to: /space/home/rosales/projects/2019_Acer_nutrients/aligned_rep_seqs_BacArc_AcNu_trim.qza


In [16]:
#Construct phylogenetic tree
!qiime phylogeny fasttree \
  --i-alignment /space/home/rosales/projects/2019_Acer_nutrients/aligned_rep_seqs_BacArc_AcNu_trim.qza \
  --o-tree /space/home/rosales/projects/2019_Acer_nutrients/unrooted-tree-_BacArc_AcNu_trim.qza

Saved Phylogeny[Unrooted] to: /space/home/rosales/projects/2019_Acer_nutrients/unrooted-tree-_BacArc_AcNu_trim.qza


In [17]:
#root tree
!qiime phylogeny midpoint-root \
--i-tree  /space/home/rosales/projects/2019_Acer_nutrients/unrooted-tree-_BacArc_AcNu_trim.qza\
--o-rooted-tree /space/home/rosales/projects/2019_Acer_nutrients/rooted-tree-_BacArc_AcNu_trim.qza

Saved Phylogeny[Rooted] to: /space/home/rosales/projects/2019_Acer_nutrients/rooted-tree-_BacArc_AcNu_trim.qza


In [18]:
#use export to convert data into biom format
!qiime tools export \
--input-path /space/home/rosales/projects/2019_Acer_nutrients/table-BacArc_AcNu_trim.qza  \
--output-path /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim 

!qiime tools export \
--input-path /space/home/rosales/projects/2019_Acer_nutrients/rooted-tree-_BacArc_AcNu_trim.qza  \
--output-path /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim 

!qiime tools export \
--input-path /space/home/rosales/projects/2019_Acer_nutrients/taxonomyAcNu_trim.qza \
--output-path /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim

Exported /space/home/rosales/projects/2019_Acer_nutrients/table-BacArc_AcNu_trim.qza as BIOMV210DirFmt to directory /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim
Exported /space/home/rosales/projects/2019_Acer_nutrients/rooted-tree-_BacArc_AcNu_trim.qza as NewickDirectoryFormat to directory /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim
Exported /space/home/rosales/projects/2019_Acer_nutrients/taxonomyAcNu_trim.qza as TSVTaxonomyDirectoryFormat to directory /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim


In [19]:
#Convert from biom format to tsv
!biom  convert \
-i /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim/feature-table.biom \
--to-tsv \
-o /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim/feature-table.txt
!wc -l  /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim/feature-table.txt

2717 /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim/feature-table.txt


In [20]:
#change OTUID label
!sed -r 's/#OTU\sID/ASV/g' /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim/feature-table.txt | grep -v "#" > /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu/feature-tableAcNu_trim.txt



In [21]:
#remove confidence column and header
%cd /space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim/
!awk -v OFS='\t' '{print $1, $2}' taxonomy.tsv | grep -v "Feature" > taxonomyAcNu_trim.tsv
!wc -l taxonomyAcNu_trim.tsv

/space/home/rosales/projects/2019_Acer_nutrients/exported_AcNu_trim
2926 taxonomyAcNu_trim.tsv
